<a href="https://colab.research.google.com/github/zahra-zarrabi/Bot_DeepLearning/blob/master/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

In [3]:
!unzip -qq face-mask-12k-images-dataset.zip

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
batch_size = 32
width=height=224
epochs=10

In [6]:
data_generator = ImageDataGenerator(rescale=1./255,
                                    horizontal_flip=True)

train_data=data_generator.flow_from_directory('/content/Face Mask Dataset/Train',
                                              batch_size=batch_size,
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=(width,height))


test_data=data_generator.flow_from_directory('/content/Face Mask Dataset/Test',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False,
                                              target_size=(width,height))


val_data=data_generator.flow_from_directory('/content/Face Mask Dataset/Validation',
                                              batch_size=batch_size,
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=(width,height))

Found 10000 images belonging to 2 classes.
Found 992 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [7]:
base_model =tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_shape=(width,height,3),
    pooling='evg',
)

102883328/102869336 [==============================] - 1s 0us/step


In [8]:
for layer in base_model.layers[:-6]:
  layer.trainable=False

In [9]:
model = tf.keras.Sequential([
                             base_model,
                             layers.Dense(2,activation='softmax')
])

In [10]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy']
             )

In [11]:
model.fit(train_data,
          batch_size=batch_size,
          validation_data=val_data,
          epochs=5)

Epoch 1/5
313/313 [==============================] - 102s 214ms/step - loss: 0.4813 - accuracy: 0.9931 - val_loss: 0.3720 - val_accuracy: 0.9875
Epoch 2/5
313/313 [==============================] - 66s 212ms/step - loss: 0.2978 - accuracy: 0.9951 - val_loss: 0.2418 - val_accuracy: 0.9950
Epoch 3/5
313/313 [==============================] - 66s 212ms/step - loss: 0.2008 - accuracy: 0.9967 - val_loss: 0.1668 - val_accuracy: 0.9975
Epoch 4/5
313/313 [==============================] - 66s 211ms/step - loss: 0.1422 - accuracy: 0.9977 - val_loss: 0.1219 - val_accuracy: 0.9975
Epoch 5/5
313/313 [==============================] - 66s 212ms/step - loss: 0.1055 - accuracy: 0.9983 - val_loss: 0.0878 - val_accuracy: 1.0000


In [ ]:
model.save('model.h5')

In [13]:
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
y_pred=model.predict(test_data)
y_pred=np.argmax(y_pred,axis=1)

print('Confusion Matrix')
print(confusion_matrix(test_data.classes,y_pred))

target_names=list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes,y_pred,target_names=target_names))

Confusion Matrix
[[483   0]
 [  3 506]]
Classification Report
              precision    recall  f1-score   support

    WithMask       0.99      1.00      1.00       483
 WithoutMask       1.00      0.99      1.00       509

    accuracy                           1.00       992
   macro avg       1.00      1.00      1.00       992
weighted avg       1.00      1.00      1.00       992

